<a href="https://colab.research.google.com/github/HealLink/ML-Model/blob/ryan's-experiment/notebook_bert_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install tensorflow[and-cuda]
%pip install -U "tensorflow-text==2.13.*"
%pip install "tf-models-official==2.13.*"

In [1]:
# Import-import
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection
import sklearn.utils
import re
import csv
import json

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

tf.get_logger().setLevel('ERROR')

2024-12-08 12:15:25.458047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733634925.524935  422101 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733634925.547047  422101 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 12:15:25.708960: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   IN_COLAB = True
else:
   print("NOT in Colab")
   IN_COLAB = False

NOT in Colab


In [4]:
if IN_COLAB:
    # Load the Drive helper and mount
    from google.colab import drive # type: ignore
    drive.mount('/content/drive')
    data_dir = "drive/MyDrive/data"
else:
    data_dir = "../data"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# Step 1: Data cleaning

In [ ]:
# Download raw dataset
!wget -O {data_dir+"/combined_data.csv"} "https://drive.google.com/uc?export=download&id=1GJn2kEIBgto2OyD7-h2HQOv_NJUriqJh"

In [ ]:
df = pd.read_csv(data_dir+"/combined_data.csv", index_col=0)

In [ ]:
df.head()

In [ ]:
df.info()

### Step 1a: Basic cleaning

In [ ]:
# Check for missing values
print(f"Number of rows with missing values: {df.isnull().sum()}")

# Check for duplicates
print(f"Number of duplicate rows: {df.duplicated(subset=['statement']).sum()}")

In [ ]:
# Drop rows that contain empty values
df = df.dropna()

# Drop rows that contain duplicate values in the ‘statement’ column and keep only the first row
df = df.drop_duplicates(subset=['statement'], keep='first')

df.reset_index(drop = True, inplace = True)

In [ ]:
# Recheck for missing values
print(f"Number of rows with missing values: {df.isnull().sum()}")

# Recheck for duplicates
print(f"Number of duplicate rows: {df.duplicated(subset=['statement']).sum()}")

### Step 1b: Deep cleaning

In [ ]:
# Change the data type of ‘statement’ and ‘status’ columns to string
df = df.astype({"statement":str, "status":str})

In [ ]:
def clean_text(text):
    text = text.lower()
    # remove stopwords
    # for word in stopwords:
    #     if word[0] == "'":
    #         text = re.sub(rf"{word}\b", "", text)
    #     else:
    #         text = re.sub(rf"\b{word}\b", "", text)

    # text = re.sub(r'[!"“’#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ' ', text) # remove punctuation mark
    # text = re.sub(emoj, ' ', text) # remove emoji
    text = re.sub(r'\d+', ' ', text) # remove number
    # text = re.sub(r'(.)\1+', r'\1', text) # remove repeated character
    # text = re.sub(r' [a-z] ', ' ', text) # remove single character
    text = re.sub(r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)", " ", text) # remove links
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces
    return text.strip()

In [ ]:
# CLEAN!!!
df['statement'] = df['statement'].apply(clean_text)
df = df[df['statement'] != ""]

### Step 1c: Very deep cleaning

In [ ]:
# Data distribution analysis of each label
df.status.value_counts()

In [ ]:
# Adding word count column for further analysis
df['word_count'] = df['statement'].apply(lambda x: len(x.split()))

In [ ]:
# Define bins and labels for word count ranges
bins = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, float('inf')]  # Adjust as needed
labels = ['1-100', '101-200', '201-300', '301-400', '401-500', '501-600', '601-700', '701-800', '801-900', '901-1000', '+1000']

# Add a column to categorize statements into ranges
df['word_count_range'] = pd.cut(df['word_count'], bins=bins, labels=labels, right=True)

In [ ]:
# Count the number of statements in each range
df['word_count_range'].value_counts().sort_index()

In [ ]:
# Group by word count range and label, then count occurrences
df.groupby(['word_count_range', 'status']).size().unstack(fill_value=0)

In [ ]:
df_correct_size = df[(df['word_count'] >= 10) & (df['word_count'] <= 192)].reset_index(drop=True)
df_correct_size.status.value_counts()

In [ ]:
df_long_statement = df[(df['word_count'] > 200)].reset_index(drop=True)
df_long_statement.status.value_counts()

In [ ]:
def sliding_window_split(df, max_length, overlap_ratio=0.5):    
    def split_sentence(sentence, max_length):
        words = sentence.split()  # Split by spaces, assuming words are separated by spaces
        chunks = []
        step = int(max_length * (1 - overlap_ratio))  # Calculate the step size based on overlap ratio
        
        # Slide over the sentence in chunks of max_length with overlap
        for i in range(0, len(words), step):
            chunk = words[i:i + max_length]
            chunks.append(' '.join(chunk))  # Join back to a sentence
            if i + max_length >= len(words):  # Exit condition if the chunk exceeds the sentence
                break
        return chunks

    expanded_rows = []

    # Iterate over the original DataFrame
    for idx, row in df.iterrows():
        sentence = row['statement']
        label = row['status']
        
        # Split the sentence into smaller chunks
        chunks = split_sentence(sentence, max_length, )
        
        # Create new rows for each chunk
        for chunk in chunks:
            expanded_rows.append({'statement': chunk, 'status': label})
    
    # Return the expanded dataframe
    return pd.DataFrame(expanded_rows)

In [ ]:
df_export_candidate = sliding_window_split(df, 192, 0.5)
df_export_candidate

In [ ]:
# Count the number of examples for each label
label_counts = df_export_candidate['status'].value_counts()

# Find the label with the minimum count
min_label = label_counts.idxmin()
min_count = label_counts.min()

print(f"Label with the lowest number of examples: {min_label}")
print(f"Number of examples: {min_count}")

In [ ]:
df_export_candidate.status.value_counts()

In [ ]:
# Optional, export the cleaned dataset
df_export_candidate.to_csv(data_dir+'/cleaned_data_10-192_links_number_removed_sliding50percent.csv', index=False)

In [ ]:
del df
del df_correct_size
del df_long_statement
del df_export_candidate

# Step 2: Prepare model

In [5]:
CLASS_NUM = 7

SEQ_LENGTH = 256
TRAINING_RATIO = 0.8
BATCH_SIZE = 8
SEED = 45

MODEL_DROPOUT_RATE = 0.2
MODEL_L2_REGULARIZER = 0.02

EPOCHS = 5
INIT_LR = 2e-5
WARM_UP_RATIO = 0.1

## Step 2a: Choosing model

In [6]:
bert_model_name = 'bert-en-uncased-l-12-h-768-a-12'
tfhub_handle_encoder = "https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-12-h-768-a-12/2"
tfhub_handle_preprocess = "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3"

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-12-h-768-a-12/2
Preprocess model auto-selected: https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3


## Step 2b: BERT Preprocess

In [7]:
def make_bert_preprocess_model(tfhub_handle_preprocess, sentence_features, seq_length=SEQ_LENGTH):
  """Returns Model mapping string features to BERT inputs.

  Args:
    sentence_features: a list with the names of string-valued features.
    seq_length: an integer that defines the sequence length of BERT inputs.

  Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
  """

  input_segments = [
      tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
      for ft in sentence_features]

  # Tokenize the text to word pieces.
  bert_preprocess = hub.load(tfhub_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in input_segments]

  # Optional: Trim segments in a smart way to fit seq_length.
  # Simple cases (like this example) can skip this step and let
  # the next step apply a default truncation to approximately equal lengths.
  truncated_segments = segments

  # Pack inputs. The details (start/end token ids, dict of output tensors)
  # are model-dependent, so this gets loaded from the SavedModel.
  packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                          arguments=dict(seq_length=seq_length),
                          name='packer')
  model_inputs = packer(truncated_segments)
  return tf.keras.Model(input_segments, model_inputs)

In [8]:
test_preprocess_model = make_bert_preprocess_model(tfhub_handle_preprocess, ['my_input1', 'my_input2'])
test_text = [np.array(['some random test sentence']),
             np.array(['another sentence'])]
text_preprocessed = test_preprocess_model(test_text)

print('Keys           : ', list(text_preprocessed.keys()))
print('Shape Word Ids : ', text_preprocessed['input_word_ids'].shape)
print('Word Ids       : ', text_preprocessed['input_word_ids'][0, :16])
print('Shape Mask     : ', text_preprocessed['input_mask'].shape)
print('Input Mask     : ', text_preprocessed['input_mask'][0, :16])
print('Shape Type Ids : ', text_preprocessed['input_type_ids'].shape)
print('Type Ids       : ', text_preprocessed['input_type_ids'][0, :16])

del test_preprocess_model
del test_text

I0000 00:00:1733634933.428877  422101 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1733634933.433276  422101 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1753 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Keys           :  ['input_type_ids', 'input_mask', 'input_word_ids']
Shape Word Ids :  (1, 256)
Word Ids       :  tf.Tensor(
[ 101 2070 6721 3231 6251  102 2178 6251  102    0    0    0    0    0
    0    0], shape=(16,), dtype=int32)
Shape Mask     :  (1, 256)
Input Mask     :  tf.Tensor([1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0], shape=(16,), dtype=int32)
Shape Type Ids :  (1, 256)
Type Ids       :  tf.Tensor([0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0], shape=(16,), dtype=int32)


## Step 2c: Defining classifier model

In [9]:
@tf.keras.utils.register_keras_serializable('capstone')
class Classifier(tf.keras.Model):
  def __init__(self, tfhub_handle_encoder, num_classes, dropout_rate, l2):
    super(Classifier, self).__init__(name="prediction")
    self.num_classes = num_classes
    self.dropout_rate = dropout_rate
    self.l2 = l2
    self.tfhub_handle_encoder = tfhub_handle_encoder
    self.encoder = hub.KerasLayer(self.tfhub_handle_encoder, trainable=True)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dense = tf.keras.layers.Dense(num_classes, kernel_regularizer=tf.keras.regularizers.l2(l2))

  def call(self, preprocessed_text):
    encoder_outputs = self.encoder(preprocessed_text)
    pooled_output = encoder_outputs["pooled_output"]
    x = self.dropout(pooled_output)
    x = self.dense(x)
    return x
  
  def get_config(self):
    config = super(Classifier, self).get_config()
    config.update({
        "num_classes": self.num_classes,
        "dropout_rate": self.dropout_rate,
        "l2": self.l2,
        "tfhub_handle_encoder": self.tfhub_handle_encoder
    })
    return config
  
  @classmethod
  def from_config(cls, config):
    return cls(**config)

In [10]:
test_classifier_model = Classifier(tfhub_handle_encoder, 2, 0.1, 0.01)
bert_raw_result = test_classifier_model(text_preprocessed)

print(f'Shape of the output: {bert_raw_result.shape}')
print(bert_raw_result)
print(tf.sigmoid(bert_raw_result))

del test_classifier_model
del bert_raw_result

Shape of the output: (1, 2)
tf.Tensor([[ 0.4686162  -0.00454919]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.61505616 0.4988627 ]], shape=(1, 2), dtype=float32)


# Step 3: Preprocess dataset

## Step 3a: Load dataset

In [11]:
sentences = []
labels = []

with open(data_dir+'/cleaned_data_10-192_imbalanced_links_number_removed_no_sliding.csv', 'r') as csvfile:
    heading = next(csvfile)
    reader_obj = csv.reader(csvfile)
    for row in reader_obj:
        labels.append(row[1])
        sentences.append(row[0])

print(f"There're {len(sentences)} sentences")

There're 33305 sentences


## Step 3b: Split dataset

In [12]:
train_sentences, val_test_sentences, train_labels, val_test_labels = sklearn.model_selection.train_test_split(
    sentences, labels,
    train_size=TRAINING_RATIO,
    stratify=labels,
    random_state=SEED
)

val_sentences, test_sentences, val_labels, test_labels = sklearn.model_selection.train_test_split(
    val_test_sentences, val_test_labels,
    train_size=0.5,
    stratify=val_test_labels,
    random_state=SEED
)

## Step 3c: Encode label

In [13]:
classes = np.unique(train_labels)
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=classes, y=train_labels)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
class_to_index = {label: index for index, label in enumerate(classes)}

train_labels_encoded = np.array([class_to_index[label] for label in train_labels])
val_labels_encoded = np.array([class_to_index[label] for label in val_labels])
test_labels_encoded = np.array([class_to_index[label] for label in test_labels])

print("Classes:", classes)
print("Class Weights:", class_weight_dict)
print("Class mapping:", class_to_index)

Classes: ['Anxiety' 'Bipolar' 'Depression' 'Normal' 'Personality disorder' 'Stress'
 'Suicidal']
Class Weights: {0: 1.942973820462335, 1: 2.7844079841153726, 2: 0.45644390385966116, 3: 0.5755762459225335, 4: 8.150504741511165, 5: 2.338013338013338, 6: 0.6069663074925393}
Class mapping: {'Anxiety': 0, 'Bipolar': 1, 'Depression': 2, 'Normal': 3, 'Personality disorder': 4, 'Stress': 5, 'Suicidal': 6}


## Step 3d: create tf.data.Dataset

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels_encoded))
val_dataset = tf.data.Dataset.from_tensor_slices((val_sentences, val_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((val_sentences, val_labels_encoded))

train_dataset = train_dataset.map(lambda x, y : (x, tf.one_hot(y, depth=CLASS_NUM)))
val_dataset = val_dataset.map(lambda x, y : (x, tf.one_hot(y, depth=CLASS_NUM)))
test_dataset = test_dataset.map(lambda x, y : (x, tf.one_hot(y, depth=CLASS_NUM)))

print(f"There are {train_dataset.cardinality()} sentence-label pairs for training.")
print(f"There are {val_dataset.cardinality()} sentence-label pairs for validation.")
print(f"There are {test_dataset.cardinality()} sentence-label pairs for test.")

There are 26644 sentence-label pairs for training.
There are 3330 sentence-label pairs for validation.
There are 3330 sentence-label pairs for test.


## Step 3e: optimize dataset

In [15]:
# Optimize the datasets for training
train_dataset_batched = (train_dataset
                       .shuffle(train_dataset.cardinality(), seed=SEED)
                       .cache()
                       .prefetch(buffer_size=tf.data.AUTOTUNE)
                       .batch(BATCH_SIZE)
                       )

val_dataset_batched = (val_dataset
                      .cache()
                      .prefetch(buffer_size=tf.data.AUTOTUNE)
                      .batch(BATCH_SIZE)
                      )

test_dataset_batched = (test_dataset
                      .cache()
                      .prefetch(buffer_size=tf.data.AUTOTUNE)
                      .batch(BATCH_SIZE)
                      )

## Step 3c: preprocess statement using bert_preprocess

In [16]:
for statement, label in train_dataset.take(1):
    print(statement)
    print(label)

tf.Tensor(b'i have been hiding in a cave within myself, it is been a few years since i have no friends, i walked away from everything and everyone, i think this was a mechanism of defense. i became a coward, i thought nothing scared me, but one thing scares me a shit: the pain, and nowadays i am so weak that even before i feel pain i think about suicide to relieve me. i think nietzsche was right when he said, "the idea of suicide is a big one consolation, helps to apply many bad nights". ** under the cave**', shape=(), dtype=string)
tf.Tensor([0. 0. 0. 0. 0. 0. 1.], shape=(7,), dtype=float32)


2024-12-08 12:15:44.259971: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
bert_preprocess_model = make_bert_preprocess_model(tfhub_handle_preprocess, ['sentence'], SEQ_LENGTH)

train_dataset_final = train_dataset_batched.map(lambda text, label: (bert_preprocess_model(text), label))
val_dataset_final = val_dataset_batched.map(lambda text, label: (bert_preprocess_model(text), label))
test_dataset_final = test_dataset_batched.map(lambda text, label: (bert_preprocess_model(text), label))

In [18]:
# delete previous variable
del sentences
del train_sentences
del val_test_sentences
del val_sentences
del test_sentences

del labels
del train_labels
del val_test_labels
del val_labels
del test_labels

del train_dataset
del val_dataset
del test_dataset

del train_dataset_batched
del val_dataset_batched
del test_dataset_batched

# Step 4: How To Train Your ~~Dragon~~ ML Model

## Step 4a: preparing classifier_model

In [19]:
classifier_model = Classifier(tfhub_handle_encoder, CLASS_NUM, MODEL_DROPOUT_RATE, MODEL_L2_REGULARIZER)
bert_raw_result = classifier_model(text_preprocessed)
print(tf.nn.softmax(bert_raw_result))

tf.Tensor(
[[0.04075588 0.16310142 0.4962448  0.15984029 0.07474155 0.04117482
  0.02414133]], shape=(1, 7), dtype=float32)


In [20]:
@tf.keras.utils.register_keras_serializable('capstone')
class MatthewsCorrelationCoefficient(tf.keras.metrics.Metric):
    def __init__(self, num_classes, dtype=None, name="matthews_correlation_coefficient"):
        super(MatthewsCorrelationCoefficient, self).__init__(name=name)
        self.num_classes = num_classes
        self.c = self.add_weight(name="total_correct_predicted", initializer="zeros", dtype=tf.float32)
        self.s = self.add_weight(name="total_samples", initializer="zeros", dtype=tf.float32)
        self.t = self.add_weight(name="num_true", shape=(num_classes,), initializer="zeros", dtype=tf.float32)
        self.p = self.add_weight(name="num_pred", shape=(num_classes,), initializer="zeros", dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert one-hot labels to integer labels
        if len(y_true.shape) > 1:
            y_true = tf.argmax(y_true, axis=-1)
        if len(y_pred.shape) > 1:
            y_pred = tf.argmax(y_pred, axis=-1)
        
        # Total number of samples
        new_s = tf.cast(tf.size(y_true), tf.float32)
        # Total number of correctly predicted labels
        new_c = tf.reduce_sum(tf.cast(tf.equal(y_true, y_pred), tf.float32))
        
        # Update state
        self.s.assign(tf.add(self.s, new_s))
        self.c.assign(tf.add(self.c, new_c))

        for k in range(self.num_classes):
            k = tf.cast(k, y_true.dtype)
            tk = tf.reduce_sum(tf.cast(tf.equal(y_true, k), tf.float32))
            pk = tf.reduce_sum(tf.cast(tf.equal(y_pred, k), tf.float32))
            self.t[k].assign(tf.add(self.t[k], tk))
            self.p[k].assign(tf.add(self.p[k], pk))
    
    def result(self):
        num = self.c * self.s - tf.reduce_sum(self.t * self.p)
        denom_t = tf.reduce_sum(self.t * self.t)
        denom_p = tf.reduce_sum(self.p * self.p)
        denom = tf.sqrt((self.s**2 - denom_t) * (self.s**2 - denom_p))
        
        mcc = tf.divide(num, denom + 1e-6)
        return mcc
    
    def reset_state(self):
        self.c.assign(0)
        self.s.assign(0)
        self.t.assign(tf.zeros_like(self.t))
        self.p.assign(tf.zeros_like(self.p))
    
    def get_config(self):
        config = super(MatthewsCorrelationCoefficient, self).get_config()
        config.update({
            "num_classes": self.num_classes
            })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
loss = tf.keras.losses.CategoricalFocalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.CategoricalAccuracy()]

steps_per_epoch = tf.data.experimental.cardinality(train_dataset_final).numpy()
num_train_steps = steps_per_epoch * EPOCHS
num_warmup_steps = int(WARM_UP_RATIO * num_train_steps)

optimizer = optimization.create_optimizer(
        init_lr=INIT_LR,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        optimizer_type='adamw'
    )

In [21]:
classifier_model.compile(optimizer='adam',
                         loss='categorical_crossentropy',
                         metrics='categorical_accuracy')

In [22]:
classifier_model.summary()

Model: "prediction"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  multiple                  109482241 
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  5383      
                                                                 
Total params: 109487624 (417.66 MB)
Trainable params: 109487623 (417.66 MB)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________


In [23]:
# Check model compatibility
example_batch = train_dataset_final.take(1)

try:
	classifier_model.evaluate(example_batch, verbose=False)
except:
	print("Your model is not compatible with the dataset you defined earlier. Check that the loss function and last layer are compatible with one another.")
else:
	predictions = classifier_model.predict(example_batch, verbose=False)
	print(f"predictions have shape: {predictions.shape}")

2024-12-08 12:16:49.279633: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


predictions have shape: (8, 7)


2024-12-08 12:16:51.450938: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Step 4b: Preparing callbacks

In [ ]:
base_checkpoint = data_dir+'/checkpoint'

In [ ]:
# Run thif if you don't determine to load existing checkpoint
checkpoint_path = base_checkpoint
counter = 1

while os.path.exists(checkpoint_path):
    checkpoint_path = f"{base_checkpoint}_{counter}"
    counter += 1

os.makedirs(checkpoint_path)

checkpoint_model_filepath = checkpoint_path+"/checkpoint_{epoch:02d}.tf"
checkpoint_num_epoch_filepath = checkpoint_path+"/current_epoch.txt"
training_log_filepath = checkpoint_path+"/training_log.json"

In [ ]:
# Run this if you want to load certain checkpoint
checkpoint_path = f"{data_dir}/checkpoint_6" # customize this line

checkpoint_model_filepath = checkpoint_path+"/checkpoint_{epoch:02d}.tf"
checkpoint_num_epoch_filepath = checkpoint_path+"/current_epoch.txt"
training_log_filepath = checkpoint_path+"/training_log.json"

In [ ]:
# Callbacks
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_model_filepath)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self, epoch_file, log_file):
        self.epoch_file = epoch_file
        self.log_file = log_file
        self.history = {"epoch": [], "loss": [], "categorical_accuracy": [], "matthews_correlation_coefficient": [],
                        "val_loss": [], "val_categorical_accuracy": [], "val_matthews_correlation_coefficient": []}

        # Load history and starting epoch if they exist
        if os.path.exists(self.log_file):
            with open(self.log_file, 'r') as f:
                self.history = json.load(f)

        if os.path.exists(self.epoch_file):
            with open(self.epoch_file, 'r') as f:
                self.starting_epoch = int(f.read())
        else:
            with open(self.epoch_file, 'w') as f:
                f.write(str(0))
            self.starting_epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        current_epoch = self.starting_epoch + epoch + 1
        # Save the current epoch
        with open(self.epoch_file, 'w') as f:
            f.write(str(current_epoch))

        # Save logs (loss, accuracy, etc.) for plotting
        self.history["epoch"].append(current_epoch)
        self.history["loss"].append(logs.get("loss"))
        self.history["categorical_accuracy"].append(logs.get("categorical_accuracy"))
        self.history["matthews_correlation_coefficient"].append(logs.get("matthews_correlation_coefficient"))
        self.history["val_loss"].append(logs.get("val_loss"))
        self.history["val_categorical_accuracy"].append(logs.get("val_categorical_accuracy"))
        self.history["val_matthews_correlation_coefficient"].append(logs.get("val_matthews_correlation_coefficient"))

        # Save history to the log file
        with open(self.log_file, 'w') as f:
            json.dump(self.history, f, indent=4)

custom_checkpoint_callback = CustomCheckpointCallback(checkpoint_num_epoch_filepath, training_log_filepath)

In [ ]:
# Load checkpoint
checkpoint_folder_name = "checkpoint_7"
try:
    with open(f"{data_dir}/{checkpoint_folder_name}/current_epoch.txt", 'r') as f:
        start_epoch = int(f.read())
    print(f"Resuming training from epoch {start_epoch}")
except FileNotFoundError:
    start_epoch = 0
    print("No saved epoch found. Starting from epoch 0")

# Load saved model
if start_epoch > 0:
    classifier_model = tf.keras.models.load_model(f"{data_dir}/{checkpoint_folder_name}/checkpoint_{start_epoch:02d}.tf", custom_objects={
                                                                                                "Classifier": Classifier, 
                                                                                                "AdamWeightDecay": optimization.AdamWeightDecay,
                                                                                                "WarmUp": optimization.WarmUp,
                                                                                                "MatthewsCorrelationCoefficient": MatthewsCorrelationCoefficient,
                                                                                            })
    print(f"Loaded model from {checkpoint_folder_name}")

## Step 4c: TRAIN!!!

In [ ]:
history = classifier_model.fit(
    train_dataset_final,
    epochs=EPOCHS,
    initial_epoch=start_epoch,
    validation_data=val_dataset_final,
    class_weight=dict(enumerate(class_weights)),
    callbacks=[
        model_checkpoint_callback,
        custom_checkpoint_callback,
        early_stopping_callback
    ]
    )

## Step 4d: EVALUATE!!!

In [ ]:
loss, mcc, accuracy = classifier_model.evaluate(test_dataset_final)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')
print(f'MCC: {mcc}')

In [ ]:
# Get predictions on the test set
y_pred_probs = classifier_model.predict(test_dataset_final)

# Convert probabilities to class indices
y_pred = np.argmax(y_pred_probs, axis=1)

# Get true labels from the test dataset
y_true = np.argmax(np.concatenate([y for x, y in test_dataset_final], axis=0), axis=-1)

# Optional: Normalize the confusion matrix
cm_normalized = confusion_matrix(y_true, y_pred, normalize='true')

cm_normalized_short= np.around(cm_normalized.astype('float') / cm_normalized.sum(axis=1)[:, np.newaxis], decimals=3)

# Plot the confusion matrix
plt.figure(figsize=(8, 8))
plt.imshow(cm_normalized, cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')
plt.colorbar()

# Add labels to each cell
for i in range(cm_normalized_short.shape[0]):
    for j in range(cm_normalized_short.shape[1]):
        plt.text(j, i, cm_normalized_short[i, j], ha='center', va='center', color='red')

plt.show()

In [ ]:
def plot_graphs(training_log):
    # Load the log file
    with open(training_log, "r") as f:
        history = json.load(f)
    
    # Plot loss
    plt.figure(figsize=(10, 5))
    plt.plot(history["epoch"], history["loss"], label="Training Loss")
    plt.plot(history["epoch"], history["val_loss"], label="Validation Loss")
    plt.title("Loss per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    # Plot accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(history["epoch"], history["matthews_correlation_coefficient"], label="Training MCC")
    plt.plot(history["epoch"], history["val_matthews_correlation_coefficient"], label="Validation MCC")
    plt.title("MCC per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("MCC")
    plt.legend()
    plt.show()

    # Plot accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(history["epoch"], history["categorical_accuracy"], label="Training Accuracy")
    plt.plot(history["epoch"], history["val_categorical_accuracy"], label="Validation Accuracy")
    plt.title("Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

plot_graphs(training_log_filepath)

# Step 5: Save & Test the model

In [24]:
main_save_path = '../models'
bert_type = tfhub_handle_encoder.split('/')[-2]
saved_model_name = f'{"test_model"}_{bert_type}'

saved_model_path = os.path.join(main_save_path, saved_model_name)
preprocess_inputs = bert_preprocess_model.inputs
bert_encoder_inputs = bert_preprocess_model(preprocess_inputs)
bert_outputs = classifier_model(bert_encoder_inputs)
model_for_export = tf.keras.Model(preprocess_inputs, bert_outputs)
model_for_export.save(saved_model_path, include_optimizer=False)

In [ ]:
rawtext_test = tf.constant(["tomorrow is our presentation. i feel so nervous. what if something going wrong"])
predictions = model_for_export(rawtext_test)

# predictions will be a numpy array of shape (1, num_classes) with probabilities for each class
print(classes)
print(tf.nn.softmax(predictions))

# To get the predicted class index
predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]
print(f"Predicted class: {classes[predicted_class_index]}")